<a href="https://colab.research.google.com/github/OneFineStarstuff/Onefinebot/blob/main/Bias_Evaluation_Example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install scikit-learn pandas fairlearn

In [ ]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.datasets import fetch_openml
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from fairlearn.metrics import demographic_parity_difference, equalized_odds_difference
from sklearn.metrics import accuracy_score

class EthicalModule:
    def __init__(self, model):
        self.model = model

    def evaluate_fairness(self, X, y_true, sensitive_data):
        y_pred = self.model.predict(X)
        dpd = demographic_parity_difference(y_true, y_pred, sensitive_features=sensitive_data)
        eod = equalized_odds_difference(y_true, y_pred, sensitive_features=sensitive_data)
        accuracy = accuracy_score(y_true, y_pred)
        return {"Accuracy": accuracy, "Demographic Parity Difference": dpd, "Equalized Odds Difference": eod}

if __name__ == "__main__":
    # Load the dataset
    data = fetch_openml(data_id=1590)  # Adult Income Dataset
    X = pd.DataFrame(data.data, columns=data.feature_names)
    y = data.target

    # Identify categorical columns for one-hot encoding
    categorical_cols = X.select_dtypes(include=['object']).columns.tolist()

    # Create a column transformer for preprocessing
    preprocessor = ColumnTransformer(
        transformers=[
            ('cat', OneHotEncoder(), categorical_cols)  # Apply OneHotEncoder to categorical columns
        ],
        remainder='passthrough'  # Keep other columns unchanged (if any)
    )

    # Split the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

    # Create a pipeline with preprocessing and model fitting
    pipeline = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('classifier', RandomForestClassifier(random_state=42))
    ])

    # Fit the model using the pipeline
    pipeline.fit(X_train, y_train)

    # Extract sensitive feature for fairness evaluation (e.g., 'sex')
    sensitive_feature = X_test['sex']  # Ensure this column exists in your test set

    # Drop sensitive feature from test set for predictions
    X_test_transformed = X_test.drop(columns=['sex'])

    # Instantiate the ethical module with the fitted pipeline model
    ethical_module = EthicalModule(pipeline)

    # Evaluate fairness metrics on the test set
    fairness_metrics = ethical_module.evaluate_fairness(X_test_transformed, y_test, sensitive_feature)

    print("Fairness Metrics:", fairness_metrics)